1. Importing Admissions and Patients CSV

In [78]:
import pandas as pd

#reading the patients and admissions csv
patients = pd.read_csv("patients.csv/patients.csv")
admissions = pd.read_csv("admissions.csv/admissions.csv")

print(patients.head(10))
print(admissions.head(10))

#merging the patients.csv and admission.csv based on subject_id
df = admissions.merge(patients, on="subject_id", how="left")
print(df.head())

   subject_id gender  anchor_age  anchor_year anchor_year_group        dod
0    10000032      F          52         2180       2014 - 2016   9/9/2180
1    10000048      F          23         2126       2008 - 2010        NaN
2    10000058      F          33         2168       2020 - 2022        NaN
3    10000068      F          19         2160       2008 - 2010        NaN
4    10000084      M          72         2160       2017 - 2019  2/13/2161
5    10000102      F          27         2136       2008 - 2010        NaN
6    10000108      M          25         2163       2014 - 2016        NaN
7    10000115      M          24         2154       2017 - 2019        NaN
8    10000117      F          48         2174       2008 - 2010        NaN
9    10000161      M          60         2163       2020 - 2022        NaN
   subject_id   hadm_id        admittime         dischtime deathtime  \
0    10000032  22595853   5/6/2180 22:23    5/7/2180 17:15       NaN   
1    10000032  22841357  6/26/2

In [79]:
# reading services csv
services = pd.read_csv("services.csv/services.csv")

print(services.head())
print(services.shape)

#merging curr_service into main dataframe
df = df.merge(
    services,
    on=["subject_id", "hadm_id"],
    how="left"
)

print(df.head())
print("Missing curr_service:", df["curr_service"].isna().sum())

   subject_id   hadm_id         transfertime prev_service curr_service
0    10000032  22595853  2180-05-06 22:24:57          NaN          MED
1    10000032  22841357  2180-06-26 18:28:08          NaN          MED
2    10000032  25742920  2180-08-05 23:44:50          NaN          MED
3    10000032  29079034  2180-07-23 12:36:04          NaN          MED
4    10000068  25022803  2160-03-03 23:17:17          NaN          MED
(593071, 5)
   subject_id   hadm_id        admittime        dischtime deathtime  \
0    10000032  22595853   5/6/2180 22:23   5/7/2180 17:15       NaN   
1    10000032  22841357  6/26/2180 18:27  6/27/2180 18:49       NaN   
2    10000032  25742920   8/5/2180 23:44   8/7/2180 17:50       NaN   
3    10000032  29079034  7/23/2180 12:35  7/25/2180 17:55       NaN   
4    10000068  25022803   3/3/2160 23:16    3/4/2160 6:26       NaN   

   admission_type admit_provider_id      admission_location  \
0          URGENT            P49AFC  TRANSFER FROM HOSPITAL   
1        

2. Importing the diagnoses_icd csv and filtering only for MI patients

In [80]:
#reading the d_icd_diagnoses csv to identify MI related ICD codes
d_icd = pd.read_csv("d_icd_diagnoses.csv/d_icd_diagnoses.csv", dtype={"icd_code": "string"})

#filtering only MI-related ICD-9 and ICD-10 codes
icd9_mi = d_icd[(d_icd["icd_version"] == 9) & (d_icd["icd_code"].str.startswith("410")) & (d_icd["icd_code"].str.len() == 5)]
icd10_mi = d_icd[(d_icd["icd_version"] == 10) &(d_icd["icd_code"].str.startswith(("I21", "I22")))]

#combining ICD-9 and ICD-10 MI codes into a single list
MI_ICD_Codes = pd.concat([icd9_mi, icd10_mi])["icd_code"].unique().tolist()
print("Myocardial Infarction ICD codes:", MI_ICD_Codes)

#reading the diagnoses_icd zip file in chunks
diagnoses_icd = "diagnoses_icd.csv.gz"
mi_rows = [] #list to store MI related diagnosis rows from each chunk

for chunk in pd.read_csv(
    diagnoses_icd,
    compression="gzip",
    chunksize=200_000,  #200,000 records at a time
    dtype={"icd_code": "string"}
):
    #filtering and storing only MI related rows
    mi_chunk = chunk[chunk["icd_code"].isin(MI_ICD_Codes)]
    mi_rows.append(mi_chunk)

#combining all MI related chunks to a single dataframe
mi_diagnoses = pd.concat(mi_rows, ignore_index=True)

print("\n", "mi_diagnoses")
print(mi_diagnoses.head())
print("MI diagnosis rows:", mi_diagnoses.shape[0])
print("Unique patients:", mi_diagnoses["subject_id"].nunique())
print("Unique admissions:", mi_diagnoses["hadm_id"].nunique())
print(mi_diagnoses.shape)

df = df.merge(
    mi_diagnoses[["subject_id", "hadm_id"]].drop_duplicates(),
    on=["subject_id", "hadm_id"],
    how="inner"
)

print("\n",df.head(10))

#below is to print - remove # when needed if not it will print everytime I run it
#mi_diagnoses.to_csv("mi_diagnoses.csv", index=False)

Myocardial Infarction ICD codes: ['41000', '41001', '41002', '41010', '41011', '41012', '41020', '41021', '41022', '41030', '41031', '41032', '41040', '41041', '41042', '41050', '41051', '41052', '41060', '41061', '41062', '41070', '41071', '41072', '41080', '41081', '41082', '41090', '41091', '41092', 'I21', 'I210', 'I2101', 'I2102', 'I2109', 'I211', 'I2111', 'I2119', 'I212', 'I2121', 'I2129', 'I213', 'I214', 'I219', 'I21A', 'I21A1', 'I21A9', 'I22', 'I220', 'I221', 'I222', 'I228', 'I229']

 mi_diagnoses
   subject_id   hadm_id  seq_num icd_code  icd_version
0    10000764  27897940        2    41071            9
1    10000980  26913865        1    41071            9
2    10001492  27463908        1    41071            9
3    10002013  24760295        1    41071            9
4    10002155  23822395        1    41011            9
MI diagnosis rows: 16829
Unique patients: 13152
Unique admissions: 16537
(16829, 5)

    subject_id   hadm_id         admittime         dischtime deathtime  \
0

3. Data Cleaning

3.1 Using only the curr_service used at admission - first service per admission

In [81]:
import numpy as np

services = services.sort_values(["subject_id", "hadm_id", "transfertime"])

services_final = services.drop_duplicates(
    subset=["subject_id", "hadm_id"],
    keep="first"
)[["subject_id", "hadm_id", "curr_service"]]

if "curr_service" in df.columns:
    df = df.drop(columns=["curr_service"])

# merge clean curr_service
df = df.merge(
    services_final,
    on=["subject_id", "hadm_id"],
    how="left"
)

3.2 Recording/adding ICD code and version for eaach admission (using seq_number)

In [82]:
mi_icd = mi_diagnoses.sort_values(["subject_id","hadm_id","seq_num"])

mi_icd = mi_icd.drop_duplicates(
    subset=["subject_id","hadm_id"]
)[["subject_id","hadm_id","icd_code","icd_version"]]

df = df.merge(
    mi_icd,
    on=["subject_id","hadm_id"],
    how="left"
)

3.3 Dropping duplicate admission rows

In [83]:
df = df.drop_duplicates(
    subset=["subject_id","hadm_id"]
).reset_index(drop=True)

3.4 Removing unnecessary rows/ keeping only required columns

In [84]:
#Renaming anchor_age variable
df["age"] = df["anchor_age"]

columns = [
    "subject_id",
    "hadm_id",
    "admittime",
    "dischtime",
    "admission_type",
    "gender",
    "age",
    "curr_service",
    "icd_code",
    "icd_version"
]

df = df[columns]

3.5 Other

In [94]:
# final checks

print("\nBEFORE")
print("Rows:", df.shape[0])
print("Unique admissions:", df[["subject_id", "hadm_id"]].drop_duplicates().shape[0])

#creating final dataset
MI_finaldf = df.copy()

print("\nAFTER")
print("Rows:", MI_finaldf.shape[0])
print("Unique admissions:", MI_finaldf[["subject_id", "hadm_id"]].drop_duplicates().shape[0])

print("\nFinal shape:", MI_finaldf.shape)
print("\n", MI_finaldf.head(5))

#saving cleaned dataset
#MI_finaldf.to_csv("MI_finaldf.csv", index=False)


BEFORE
Rows: 16537
Unique admissions: 16537

AFTER
Rows: 16537
Unique admissions: 16537

Final shape: (16537, 10)

    subject_id   hadm_id         admittime         dischtime admission_type  \
0    10000764  27897940  10/14/2132 23:31  10/19/2132 16:30         URGENT   
1    10000980  26913865    6/27/2189 7:38     7/3/2189 3:00       EW EMER.   
2    10001492  27463908   9/23/2136 18:02   9/25/2136 17:45       EW EMER.   
3    10002013  24760295   7/10/2160 19:33   7/12/2160 12:30       EW EMER.   
4    10002155  23822395    8/4/2129 12:44   8/18/2129 16:53       EW EMER.   

  gender  age curr_service icd_code  icd_version  
0      M   86          MED    41071            9  
1      F   73          MED    41071            9  
2      F   71          MED    41071            9  
3      F   53         CMED    41071            9  
4      F   80         CMED    41011            9  


4. Data Engineering/ Variable creation

4.1 Creating Length of Stay Category (<7 or >7 days)

In [95]:
#converting admission and discharge time to datetime format
MI_finaldf["admittime"] = pd.to_datetime(MI_finaldf["admittime"])
MI_finaldf["dischtime"] = pd.to_datetime(MI_finaldf["dischtime"])

#calculating length of stay (LOS) in days
MI_finaldf["los_days"] = (
    (MI_finaldf["dischtime"] - MI_finaldf["admittime"])
    .dt.total_seconds() / (60 * 60 * 24)
)
MI_finaldf["los_days"] = MI_finaldf["los_days"].round(1)

#creating LOS categories:
MI_finaldf["los_cat"] = np.where(
    MI_finaldf["los_days"] < 7,
    "< 7 days",
    "≥ 7 days"
)
print(MI_finaldf[["admittime", "dischtime", "los_days", "los_cat"]].head())

            admittime           dischtime  los_days   los_cat
0 2132-10-14 23:31:00 2132-10-19 16:30:00       4.7  < 7 days
1 2189-06-27 07:38:00 2189-07-03 03:00:00       5.8  < 7 days
2 2136-09-23 18:02:00 2136-09-25 17:45:00       2.0  < 7 days
3 2160-07-10 19:33:00 2160-07-12 12:30:00       1.7  < 7 days
4 2129-08-04 12:44:00 2129-08-18 16:53:00      14.2  ≥ 7 days


4.2 Weekend effect (Weekend/Weekday flag)

In [96]:
#creating day-of-week variable from admission time (Monday=0, Sunday=6) - check
MI_finaldf["admit_dayofweek"] = MI_finaldf["admittime"].dt.dayofweek

#creating weekday vs weekend tag
MI_finaldf["admit_weekend"] = MI_finaldf["admit_dayofweek"].apply(
    lambda x: "Weekend" if x >= 5 else "Weekday"
)

#weekday vs weekend count check
print(MI_finaldf["admit_weekend"].value_counts())

#quick check with actual day names
tmp = MI_finaldf[["admittime"]].copy()
tmp["day_name"] = MI_finaldf["admittime"].dt.day_name()
tmp["dayofweek"] = MI_finaldf["admittime"].dt.dayofweek
print("\n",tmp.head(15))

admit_weekend
Weekday    11772
Weekend     4765
Name: count, dtype: int64

              admittime   day_name  dayofweek
0  2132-10-14 23:31:00    Tuesday          1
1  2189-06-27 07:38:00   Saturday          5
2  2136-09-23 18:02:00     Sunday          6
3  2160-07-10 19:33:00   Thursday          3
4  2129-08-04 12:44:00   Thursday          3
5  2141-05-22 20:17:00     Monday          0
6  2187-02-23 16:01:00     Friday          4
7  2111-01-02 19:21:00     Friday          4
8  2166-02-15 13:06:00   Saturday          5
9  2125-06-23 18:37:00   Saturday          5
10 2132-12-12 01:43:00     Friday          4
11 2111-11-13 23:39:00     Friday          4
12 2167-11-07 19:05:00   Saturday          5
13 2115-12-11 05:24:00  Wednesday          2
14 2111-04-12 17:51:00     Sunday          6


4.3 Prior MI admissions

In [97]:
#checking for prior MI admissions
MI_finaldf = MI_finaldf.sort_values(["subject_id", "admittime"]).reset_index(drop=True)
MI_finaldf["prior_mi"] = MI_finaldf.groupby("subject_id").cumcount().gt(0).map({True: "Y", False: "N"})

#Quick check
print(MI_finaldf["prior_mi"].value_counts(dropna=False))
print(MI_finaldf.head())

prior_mi
N    13152
Y     3385
Name: count, dtype: int64
   subject_id   hadm_id           admittime           dischtime  \
0    10000764  27897940 2132-10-14 23:31:00 2132-10-19 16:30:00   
1    10000980  26913865 2189-06-27 07:38:00 2189-07-03 03:00:00   
2    10001492  27463908 2136-09-23 18:02:00 2136-09-25 17:45:00   
3    10002013  24760295 2160-07-10 19:33:00 2160-07-12 12:30:00   
4    10002155  23822395 2129-08-04 12:44:00 2129-08-18 16:53:00   

  admission_type gender  age curr_service icd_code  icd_version  los_days  \
0         URGENT      M   86          MED    41071            9       4.7   
1       EW EMER.      F   73          MED    41071            9       5.8   
2       EW EMER.      F   71          MED    41071            9       2.0   
3       EW EMER.      F   53         CMED    41071            9       1.7   
4       EW EMER.      F   80         CMED    41011            9      14.2   

    los_cat  admit_dayofweek admit_weekend prior_mi  
0  < 7 days            

4.4 Other Diagnoses at admission

In [ ]:
hadm_diag_counts = {}

for chunk in pd.read_csv(
    "diagnoses_icd.csv.gz",
    compression="gzip",
    chunksize=200_000,
    usecols=["hadm_id", "icd_code"],
    dtype={"icd_code": "string"}
):

    #excluding MI ICD codes
    chunk = chunk[~chunk["icd_code"].isin(MI_ICD_Codes)]

    counts = chunk["hadm_id"].value_counts()

    for hadm_id, count in counts.items():
        hadm_diag_counts[hadm_id] = hadm_diag_counts.get(hadm_id, 0) + count


MI_finaldf["num_diagnoses_at_admission"] = (
    MI_finaldf["hadm_id"]
    .map(hadm_diag_counts)
    .fillna(0)
    .astype(int)
)

# quick check
print(MI_finaldf.head())

   subject_id   hadm_id           admittime           dischtime  \
0    10000764  27897940 2132-10-14 23:31:00 2132-10-19 16:30:00   
1    10000980  26913865 2189-06-27 07:38:00 2189-07-03 03:00:00   
2    10001492  27463908 2136-09-23 18:02:00 2136-09-25 17:45:00   
3    10002013  24760295 2160-07-10 19:33:00 2160-07-12 12:30:00   
4    10002155  23822395 2129-08-04 12:44:00 2129-08-18 16:53:00   

  admission_type gender  age curr_service icd_code  icd_version  los_days  \
0         URGENT      M   86          MED    41071            9       4.7   
1       EW EMER.      F   73          MED    41071            9       5.8   
2       EW EMER.      F   71          MED    41071            9       2.0   
3       EW EMER.      F   53         CMED    41071            9       1.7   
4       EW EMER.      F   80         CMED    41011            9      14.2   

    los_cat  admit_dayofweek admit_weekend prior_mi  \
0  < 7 days                1       Weekday        N   
1  < 7 days             

In [106]:
#quick check

example_id = 27897940

count = 0
for chunk in pd.read_csv(
    "diagnoses_icd.csv.gz",
    compression="gzip",
    chunksize=200_000,
    usecols=["hadm_id"]
):
    count += (chunk["hadm_id"] == example_id).sum()

print(count)

19


4.5 Readmission group

In [107]:
#sorting by patient and admission time and getting the next admission time 
MI_finaldf = MI_finaldf.sort_values(["subject_id", "admittime"]).reset_index(drop=True)
MI_finaldf["next_admittime"] = MI_finaldf.groupby("subject_id")["admittime"].shift(-1)

#calculating days from discharge to next MI admission
MI_finaldf["days_to_readmit"] = (MI_finaldf["next_admittime"] - MI_finaldf["dischtime"]).dt.days

#readmission risk categories
MI_finaldf["readmission_risk"] = np.select(
    [
        MI_finaldf["days_to_readmit"].between(0, 30, inclusive="both"),
        MI_finaldf["days_to_readmit"].between(31, 60, inclusive="both"),
        (MI_finaldf["days_to_readmit"] > 60) | (MI_finaldf["days_to_readmit"].isna())
    ],
    ["High", "Medium", "Low"],
    default="Low"
)

#quick check
print(MI_finaldf["readmission_risk"].value_counts())
print("\n", MI_finaldf.head())


MI_finaldf.to_csv("MI_final_dataset.csv", index=False)

print(MI_finaldf.shape)

readmission_risk
Low       14857
High       1420
Medium      260
Name: count, dtype: int64

    subject_id   hadm_id           admittime           dischtime  \
0    10000764  27897940 2132-10-14 23:31:00 2132-10-19 16:30:00   
1    10000980  26913865 2189-06-27 07:38:00 2189-07-03 03:00:00   
2    10001492  27463908 2136-09-23 18:02:00 2136-09-25 17:45:00   
3    10002013  24760295 2160-07-10 19:33:00 2160-07-12 12:30:00   
4    10002155  23822395 2129-08-04 12:44:00 2129-08-18 16:53:00   

  admission_type gender  age curr_service icd_code  icd_version  los_days  \
0         URGENT      M   86          MED    41071            9       4.7   
1       EW EMER.      F   73          MED    41071            9       5.8   
2       EW EMER.      F   71          MED    41071            9       2.0   
3       EW EMER.      F   53         CMED    41071            9       1.7   
4       EW EMER.      F   80         CMED    41011            9      14.2   

    los_cat  admit_dayofweek admit_weeken

5. Modelling and Evaluation

5.1 Setting predictors and targets - creating a Data frame for Modelling 

In [122]:
#predictors
predictor_cols = [
    "age",
    "gender",
    "admission_type",
    "curr_service",
    "admit_weekend",
    "prior_mi",
    "num_diagnoses_at_admission"
]

# targets
target_cols = [
    "los_cat",
    "readmission_risk"
]

#building a separate dataframe for modelling
Finalmodel_df = MI_finaldf[predictor_cols + target_cols + ["subject_id"]].copy()

#quick checks
print("Rows:", Finalmodel_df.shape[0])
print("Columns:", Finalmodel_df.shape[1])
print("\nMissing values per column:")
print(Finalmodel_df.isna().sum())

print("\nTarget distributions:")
print("\nLOS category:")
print(Finalmodel_df["los_cat"].value_counts(dropna=False))
print('\n', Finalmodel_df["readmission_risk"].value_counts(dropna=False))
print(Finalmodel_df.head(10))

#Finalmodel_df.to_csv("Finalmodel_df.csv", index=False, encoding="utf-8-sig")
#Finalmodel_df.to_csv(r"C:\Users\User\Downloads\MI_final_model_dataset.csv", index=False, encoding="utf-8-sig")

Rows: 16537
Columns: 10

Missing values per column:
age                           0
gender                        0
admission_type                0
curr_service                  2
admit_weekend                 0
prior_mi                      0
num_diagnoses_at_admission    0
los_cat                       0
readmission_risk              0
subject_id                    0
dtype: int64

Target distributions:

LOS category:
los_cat
< 7 days    10053
≥ 7 days     6484
Name: count, dtype: int64

 readmission_risk
Low       14857
High       1420
Medium      260
Name: count, dtype: int64
   age gender     admission_type curr_service admit_weekend prior_mi  \
0   86      M             URGENT          MED       Weekday        N   
1   73      F           EW EMER.          MED       Weekend        N   
2   71      F           EW EMER.          MED       Weekend        N   
3   53      F           EW EMER.         CMED       Weekday        N   
4   80      F           EW EMER.         CMED       We

In [123]:
Finalmodel_df["curr_service"] = Finalmodel_df["curr_service"].fillna("UNKNOWN")

5.2 Creating Train Test split

In [124]:
from sklearn.model_selection import train_test_split

#spliting unique patients into train and test dfs (80/20)
train_patients, test_patients = train_test_split(Finalmodel_df["subject_id"].unique(), test_size=0.2, random_state=42)

#train/test dataframes using patient split
train_df = Finalmodel_df[Finalmodel_df["subject_id"].isin(train_patients)].copy()
test_df  = Finalmodel_df[Finalmodel_df["subject_id"].isin(test_patients)].copy()

#dropping subject_id from features and creating x y variables for LOS and readmission
X_train = train_df.drop(columns=["los_cat", "readmission_risk", "subject_id"])
y_train_los, y_train_readmit = train_df["los_cat"], train_df["readmission_risk"]
X_test  = test_df.drop(columns=["los_cat", "readmission_risk", "subject_id"])
y_test_los,  y_test_readmit  = test_df["los_cat"],  test_df["readmission_risk"]

#quick checks
print("Train rows:", X_train.shape[0], "| Test rows:", X_test.shape[0])
print("Train patients:", train_df["subject_id"].nunique(), "| Test patients:", test_df["subject_id"].nunique())
print("Patient overlap:", len(set(train_df["subject_id"]) & set(test_df["subject_id"])))

Train rows: 13256 | Test rows: 3281
Train patients: 10521 | Test patients: 2631
Patient overlap: 0


5.3 Encoding variables

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack

# 1) define categorical + numeric columns (UPDATED)
cat_cols = ["gender","admission_type","curr_service","admit_weekend","prior_mi"]
num_cols = ["age", "num_diagnoses_at_admission"]

encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=True)

X_train_cat = encoder.fit_transform(X_train[cat_cols])
X_test_cat  = encoder.transform(X_test[cat_cols])

#numeric features
X_train_num = X_train[num_cols].astype(float).values
X_test_num  = X_test[num_cols].astype(float).values

#combining categorical and numeric features into final matrices
X_train_enc = hstack([X_train_cat, X_train_num])
X_test_enc  = hstack([X_test_cat, X_test_num])

print("X_train_enc:", X_train_enc.shape, "| X_test_enc:", X_test_enc.shape)

#quick checks
print("\nNum features:", num_cols)
print("\nFirst 20 OHE features:", encoder.get_feature_names_out(cat_cols))

print("\nTrain LOS distribution:")
print(y_train_los.value_counts(normalize=True))

print("\nTest LOS distribution:")
print(y_test_los.value_counts(normalize=True))

print("\nTrain Readmission distribution:")
print(y_train_readmit.value_counts(normalize=True))

print("\nTest Readmission distribution:")
print(y_test_readmit.value_counts(normalize=True))

X_train_enc: (13256, 36) | X_test_enc: (3281, 36)

Num features: ['age', 'num_diagnoses_at_admission']

First 20 OHE features: ['gender_F' 'gender_M' 'admission_type_AMBULATORY OBSERVATION'
 'admission_type_DIRECT EMER.' 'admission_type_DIRECT OBSERVATION'
 'admission_type_ELECTIVE' 'admission_type_EU OBSERVATION'
 'admission_type_EW EMER.' 'admission_type_OBSERVATION ADMIT'
 'admission_type_SURGICAL SAME DAY ADMISSION' 'admission_type_URGENT'
 'curr_service_CMED' 'curr_service_CSURG' 'curr_service_ENT'
 'curr_service_EYE' 'curr_service_GU' 'curr_service_GYN'
 'curr_service_MED' 'curr_service_NMED' 'curr_service_NSURG'
 'curr_service_OBS' 'curr_service_OMED' 'curr_service_ORTHO'
 'curr_service_PSURG' 'curr_service_PSYCH' 'curr_service_SURG'
 'curr_service_TRAUM' 'curr_service_TSURG' 'curr_service_UNKNOWN'
 'curr_service_VSURG' 'admit_weekend_Weekday' 'admit_weekend_Weekend'
 'prior_mi_N' 'prior_mi_Y']

Train LOS distribution:
los_cat
< 7 days    0.608856
≥ 7 days    0.391144
Name: prop

5.3 Length of Stay Modelling and Evaluation

5.3.1 Logisting Regression for LOS

In [132]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

logreg_los = LogisticRegression(max_iter=1000, random_state=42)

#training the model on training data
logreg_los.fit(X_train_enc, y_train_los)

#predictions on the test set
y_pred_los = logreg_los.predict(X_test_enc)
y_prob_los = logreg_los.predict_proba(X_test_enc)[:, 1] #(for ROC-AUC)

#evaluation
print("Accuracy (Logistic Regression - LOS):", round(accuracy_score(y_test_los, y_pred_los), 4))
print("\nClassification Report (Logistic Regression - LOS):\n", classification_report(y_test_los, y_pred_los))
print("Confusion Matrix (Logistic Regression - LOS):\n", confusion_matrix(y_test_los, y_pred_los))

roc_los = roc_auc_score((y_test_los == "≥ 7 days").astype(int), y_prob_los)
print("ROC-AUC (Logistic Regression - LOS):", round(roc_los, 4))

Accuracy (Logistic Regression - LOS): 0.7577

Classification Report (Logistic Regression - LOS):
               precision    recall  f1-score   support

    < 7 days       0.77      0.86      0.81      1982
    ≥ 7 days       0.73      0.61      0.67      1299

    accuracy                           0.76      3281
   macro avg       0.75      0.73      0.74      3281
weighted avg       0.76      0.76      0.75      3281

Confusion Matrix (Logistic Regression - LOS):
 [[1695  287]
 [ 508  791]]
ROC-AUC (Logistic Regression - LOS): 0.8278


5.3.2 Decision Tree for LOS

In [133]:
from sklearn.tree import DecisionTreeClassifier

decisiontree_los = DecisionTreeClassifier(max_depth=5, class_weight="balanced", random_state=42)

decisiontree_los.fit(X_train_enc, y_train_los)

y_pred_dt = decisiontree_los.predict(X_test_enc)
y_prob_dt = decisiontree_los.predict_proba(X_test_enc)[:, 1]

print("\n", "Accuracy (Decision Tree - LOS):", round(accuracy_score(y_test_los, y_pred_dt), 4))
print("\nClassification Report (Decision Tree - LOS):\n", classification_report(y_test_los, y_pred_dt))
print("Confusion Matrix (Decision Tree - LOS):\n", confusion_matrix(y_test_los, y_pred_dt))
roc_dt = roc_auc_score((y_test_los == "≥ 7 days").astype(int), y_prob_dt)
print("ROC-AUC (Decision Tree - LOS):", round(roc_dt, 4))


 Accuracy (Decision Tree - LOS): 0.7098

Classification Report (Decision Tree - LOS):
               precision    recall  f1-score   support

    < 7 days       0.83      0.65      0.73      1982
    ≥ 7 days       0.60      0.79      0.68      1299

    accuracy                           0.71      3281
   macro avg       0.72      0.72      0.71      3281
weighted avg       0.74      0.71      0.71      3281

Confusion Matrix (Decision Tree - LOS):
 [[1297  685]
 [ 267 1032]]
ROC-AUC (Decision Tree - LOS): 0.8122


5.3.3 Random Forest for LOS

In [136]:
from sklearn.ensemble import RandomForestClassifier

randomforest_los = RandomForestClassifier( n_estimators=200, max_depth=8, class_weight="balanced",random_state=42)

randomforest_los.fit(X_train_enc, y_train_los)

y_pred_rf = randomforest_los.predict(X_test_enc)
y_prob_rf = randomforest_los.predict_proba(X_test_enc)[:, 1]

print("\n", "Accuracy (Random Forest - LOS):", round(accuracy_score(y_test_los, y_pred_rf), 4))
print("\nClassification Report (Random Forest - LOS):\n", classification_report(y_test_los, y_pred_rf))
print("Confusion Matrix (Random Forest - LOS):\n", confusion_matrix(y_test_los, y_pred_rf))
print("ROC-AUC (Random Forest - LOS):", round(roc_auc_score((y_test_los == "≥ 7 days").astype(int), y_prob_rf), 4))


 Accuracy (Random Forest - LOS): 0.7351

Classification Report (Random Forest - LOS):
               precision    recall  f1-score   support

    < 7 days       0.83      0.70      0.76      1982
    ≥ 7 days       0.63      0.78      0.70      1299

    accuracy                           0.74      3281
   macro avg       0.73      0.74      0.73      3281
weighted avg       0.75      0.74      0.74      3281

Confusion Matrix (Random Forest - LOS):
 [[1393  589]
 [ 280 1019]]
ROC-AUC (Random Forest - LOS): 0.8182


5.3.4 Gradient Boosting for LOS

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gradientboosting_los = GradientBoostingClassifier(n_estimators=200, learning_rate=0.05, max_depth=3, random_state=42)

gradientboosting_los.fit(X_train_enc, y_train_los)

y_pred_gb = gradientboosting_los.predict(X_test_enc)
y_prob_gb = gradientboosting_los.predict_proba(X_test_enc)[:, 1]

print("Accuracy (Gradient Boosting - LOS):", round(accuracy_score(y_test_los, y_pred_gb), 4))
print("\nClassification Report (Gradient Boosting - LOS):\n", classification_report(y_test_los, y_pred_gb))
print("Confusion Matrix (Gradient Boosting - LOS):\n", confusion_matrix(y_test_los, y_pred_gb))
print("ROC-AUC (Gradient Boosting - LOS):", round(roc_auc_score((y_test_los == "≥ 7 days").astype(int), y_prob_gb), 4))

Accuracy (Gradient Boosting - LOS): 0.7571

Classification Report (Gradient Boosting - LOS):
               precision    recall  f1-score   support

    < 7 days       0.78      0.84      0.81      1982
    ≥ 7 days       0.72      0.63      0.67      1299

    accuracy                           0.76      3281
   macro avg       0.75      0.74      0.74      3281
weighted avg       0.75      0.76      0.75      3281

Confusion Matrix (Gradient Boosting - LOS):
 [[1661  321]
 [ 476  823]]
ROC-AUC (Gradient Boosting - LOS): 0.8283


5.3.5 Cat boost for LOS

In [ ]:
from catboost import CatBoostClassifier

#class weights (balanced) - without extra imports
_counts = y_train_los.value_counts()
w_short = float(_counts.max() / _counts.get("< 7 days", 1))
w_long  = float(_counts.max() / _counts.get("≥ 7 days", 1))

catboost_los = CatBoostClassifier(
    iterations=800,
    learning_rate=0.05,
    depth=6,
    loss_function="Logloss",
    class_weights={
        "< 7 days": w_short,
        "≥ 7 days": w_long
    },
    random_seed=42,
    verbose=False
)

catboost_los.fit(X_train_enc, y_train_los)

y_pred_cb = catboost_los.predict(X_test_enc)
y_prob_cb = catboost_los.predict_proba(X_test_enc)[:, 1]

print("\n", "Accuracy (CatBoost - LOS):", round(accuracy_score(y_test_los, y_pred_cb), 4))
print("\nClassification Report (CatBoost - LOS):\n", classification_report(y_test_los, y_pred_cb))
print("Confusion Matrix (CatBoost - LOS):\n", confusion_matrix(y_test_los, y_pred_cb))
print("ROC-AUC (CatBoost - LOS):", round(roc_auc_score((y_test_los == "≥ 7 days").astype(int), y_prob_cb), 4))


 Accuracy (CatBoost - LOS): 0.7354

Classification Report (CatBoost - LOS):
               precision    recall  f1-score   support

    < 7 days       0.82      0.73      0.77      1982
    ≥ 7 days       0.64      0.75      0.69      1299

    accuracy                           0.74      3281
   macro avg       0.73      0.74      0.73      3281
weighted avg       0.75      0.74      0.74      3281

Confusion Matrix (CatBoost - LOS):
 [[1438  544]
 [ 324  975]]
ROC-AUC (CatBoost - LOS): 0.8229


5.3.6 Support Vector Machine (SVM) for LOS

In [140]:
from sklearn.svm import SVC

svm_los = SVC(
    kernel="rbf",
    C=1.0,
    gamma="scale",
    class_weight="balanced",
    probability=True,
    random_state=42
)

svm_los.fit(X_train_enc, y_train_los)

y_pred_svm = svm_los.predict(X_test_enc)
y_prob_svm = svm_los.predict_proba(X_test_enc)[:, 1]

print("\n", "Accuracy (SVM - LOS):", round(accuracy_score(y_test_los, y_pred_svm), 4))
print("\nClassification Report (SVM - LOS):\n", classification_report(y_test_los, y_pred_svm))
print("Confusion Matrix (SVM - LOS):\n", confusion_matrix(y_test_los, y_pred_svm))
print("ROC-AUC (SVM - LOS):", round(roc_auc_score((y_test_los == "≥ 7 days").astype(int), y_prob_svm), 4))


 Accuracy (SVM - LOS): 0.7257

Classification Report (SVM - LOS):
               precision    recall  f1-score   support

    < 7 days       0.81      0.71      0.76      1982
    ≥ 7 days       0.63      0.75      0.68      1299

    accuracy                           0.73      3281
   macro avg       0.72      0.73      0.72      3281
weighted avg       0.74      0.73      0.73      3281

Confusion Matrix (SVM - LOS):
 [[1408  574]
 [ 326  973]]
ROC-AUC (SVM - LOS): 0.8139


In [141]:
# compare all trained models (NO extra imports needed)

models = {
    "Logistic Regression": logreg_los,
    "Decision Tree": decisiontree_los,
    "Random Forest": randomforest_los,
    "Gradient Boosting": gradientboosting_los,
    "CatBoost": catboost_los,
    "SVM": svm_los
}

comparison_results = []

for name, model in models.items():

    y_pred = model.predict(X_test_enc)
    y_prob = model.predict_proba(X_test_enc)[:, 1]

    acc = accuracy_score(y_test_los, y_pred)

    report = classification_report(y_test_los, y_pred, output_dict=True)

    precision = report["≥ 7 days"]["precision"]
    recall = report["≥ 7 days"]["recall"]
    f1 = report["≥ 7 days"]["f1-score"]

    auc = roc_auc_score((y_test_los == "≥ 7 days").astype(int), y_prob)

    comparison_results.append({
        "Model": name,
        "Accuracy": round(acc, 4),
        "Precision": round(precision, 4),
        "Recall": round(recall, 4),
        "F1-score": round(f1, 4),
        "ROC-AUC": round(auc, 4)
    })


comparison_df = pd.DataFrame(comparison_results)
comparison_df = comparison_df.sort_values("ROC-AUC", ascending=False)

print(comparison_df)

                 Model  Accuracy  Precision  Recall  F1-score  ROC-AUC
3    Gradient Boosting    0.7571     0.7194  0.6336    0.6738   0.8283
0  Logistic Regression    0.7577     0.7338  0.6089    0.6655   0.8278
4             CatBoost    0.7354     0.6419  0.7506    0.6920   0.8229
2        Random Forest    0.7351     0.6337  0.7844    0.7011   0.8182
5                  SVM    0.7257     0.6290  0.7490    0.6838   0.8139
1        Decision Tree    0.7098     0.6010  0.7945    0.6844   0.8122


Other tests we did - can use below for reference

In [7]:
#Hyperparameter tuning for LR - LOS

# --- 1) what we want to optimize: F1 for the long-stay class ("≥ 7 days")
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, f1_score

f1_long = make_scorer(f1_score, pos_label="≥ 7 days")

# --- 2) cross-validation setup (runs multiple train/validation splits inside the training set)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# --- 3) hyperparameter search space (these are the "knobs" we are tuning)
param_lr = {
    "C": [0.01, 0.05, 0.1, 0.3, 0.5, 1, 2, 5, 10],   # regularization strength
    "solver": ["liblinear", "lbfgs"]                  # optimization method
}

# --- 4) model (base settings)
lr_base = LogisticRegression(class_weight="balanced", max_iter=5000, random_state=42)

# --- 5) THE TUNING STEP happens here:
# RandomizedSearchCV tries many combinations from param_lr using CV, then keeps the best one.
lr_search = RandomizedSearchCV(
    estimator=lr_base,
    param_distributions=param_lr,
    n_iter=12,                 # increase for more search (slower); decrease for faster
    scoring=f1_long,           # what "best" means
    refit=True,                # refit best model on full training data at the end
    cv=cv,
    n_jobs=-1,
    random_state=42,
    verbose=1
)

lr_search.fit(X_train_enc, y_train_los)

# --- 6) best tuned model (this is what you'll use later / save)
best_lr = lr_search.best_estimator_

# --- 7) evaluate tuned model on test set (same print style you used)
y_pred = best_lr.predict(X_test_enc)
y_prob = best_lr.predict_proba(X_test_enc)[:, 1]
y_test_bin = (y_test_los == "≥ 7 days").astype(int)

print("\n===== Tuned Logistic Regression (LOS) =====")
print("Best Params:", lr_search.best_params_)
print("Best CV F1 (≥7 days):", round(lr_search.best_score_, 4))
print("Accuracy:", round(accuracy_score(y_test_los, y_pred), 4))
print("\nClassification Report:\n", classification_report(y_test_los, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test_los, y_pred))
print("ROC-AUC:", round(roc_auc_score(y_test_bin, y_prob), 4)) 

Fitting 5 folds for each of 12 candidates, totalling 60 fits

===== Tuned Logistic Regression (LOS) =====
Best Params: {'solver': 'liblinear', 'C': 0.01}
Best CV F1 (≥7 days): 0.505
Accuracy: 0.5285

Classification Report:
               precision    recall  f1-score   support

    < 7 days       0.65      0.47      0.55      1982
    ≥ 7 days       0.43      0.61      0.51      1299

    accuracy                           0.53      3281
   macro avg       0.54      0.54      0.53      3281
weighted avg       0.56      0.53      0.53      3281

Confusion Matrix:
 [[ 936 1046]
 [ 501  798]]
ROC-AUC: 0.5464


In [11]:
from catboost import Pool, cv as cb_cv, CatBoostClassifier

# binary target for AUC
y_train_bin = (y_train_los == "≥ 7 days").astype(int)
y_test_bin  = (y_test_los  == "≥ 7 days").astype(int)

train_pool = Pool(X_train_enc, y_train_bin)

cb_params = {
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "random_seed": 42,
    "class_weights": [w_short, w_long],
    "learning_rate": 0.05,
    "depth": 6,
    "l2_leaf_reg": 3,
    "logging_level": "Silent"   # stops fold spam
}

cv_results = cb_cv(
    pool=train_pool,
    params=cb_params,
    fold_count=5,
    iterations=1200,
    early_stopping_rounds=50,
    shuffle=True,
    stratified=True,
    verbose=False
)

best_iter = int(cv_results["test-AUC-mean"].idxmax() + 1)
best_auc  = float(cv_results["test-AUC-mean"].max())

best_cb = CatBoostClassifier(
    iterations=best_iter,
    **cb_params
)

best_cb.fit(X_train_enc, y_train_bin)

y_prob = best_cb.predict_proba(X_test_enc)[:, 1]
y_pred_bin = (y_prob >= 0.5).astype(int)
y_pred_lbl = np.where(y_pred_bin == 1, "≥ 7 days", "< 7 days")

print("\n===== CatBoost CV (LOS) =====")
print("Best Iterations:", best_iter)
print("Best CV AUC:", round(best_auc, 4))
print("Accuracy (CatBoost Tuned - LOS):", round(accuracy_score(y_test_los, y_pred_lbl), 4))
print("\nClassification Report (CatBoost Tuned - LOS):\n", classification_report(y_test_los, y_pred_lbl))
print("Confusion Matrix (CatBoost Tuned - LOS):\n", confusion_matrix(y_test_los, y_pred_lbl))
print("ROC-AUC (CatBoost Tuned - LOS):", round(roc_auc_score(y_test_bin, y_prob), 4))


===== CatBoost CV (LOS) =====
Best Iterations: 26
Best CV AUC: 0.5631
Accuracy (CatBoost Tuned - LOS): 0.5233

Classification Report (CatBoost Tuned - LOS):
               precision    recall  f1-score   support

    < 7 days       0.65      0.46      0.54      1982
    ≥ 7 days       0.43      0.62      0.51      1299

    accuracy                           0.52      3281
   macro avg       0.54      0.54      0.52      3281
weighted avg       0.56      0.52      0.53      3281

Confusion Matrix (CatBoost Tuned - LOS):
 [[ 914 1068]
 [ 496  803]]
ROC-AUC (CatBoost Tuned - LOS): 0.5456


In [12]:
# =========================
# Hyperparameter tuning (LOS) - SVM ONLY (FAST)
# =========================

param_svm = {
    "C": [0.1, 0.3, 0.5, 1, 2, 5, 10],
    "gamma": ["scale", 0.01, 0.05, 0.1],
    "kernel": ["rbf"]   # keep only rbf for speed
}

svm_base = SVC(
    class_weight="balanced",
    probability=False,   # faster tuning
    random_state=42
)

svm_search = RandomizedSearchCV(
    estimator=svm_base,
    param_distributions=param_svm,
    n_iter=12,
    scoring=f1_long,
    refit=True,
    cv=cv,
    n_jobs=-1,
    random_state=42,
    verbose=1
)

svm_search.fit(X_train_enc, y_train_los)

# retrain final SVM with probability=True for ROC-AUC
best_svm = SVC(
    **svm_search.best_params_,
    class_weight="balanced",
    probability=True,
    random_state=42
)

best_svm.fit(X_train_enc, y_train_los)

# evaluate
y_pred = best_svm.predict(X_test_enc)
y_prob = best_svm.predict_proba(X_test_enc)[:, 1]

print("\n===== Tuned SVM (LOS) =====")
print("Best Params:", svm_search.best_params_)
print("Best CV F1 (≥7 days):", round(svm_search.best_score_, 4))
print("Accuracy:", round(accuracy_score(y_test_los, y_pred), 4))
print("\nClassification Report:\n", classification_report(y_test_los, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test_los, y_pred))
print("ROC-AUC:", round(roc_auc_score(y_test_bin, y_prob), 4))

Fitting 5 folds for each of 12 candidates, totalling 60 fits

===== Tuned SVM (LOS) =====
Best Params: {'kernel': 'rbf', 'gamma': 0.01, 'C': 0.1}
Best CV F1 (≥7 days): 0.5469
Accuracy: 0.4505

Classification Report:
               precision    recall  f1-score   support

    < 7 days       0.62      0.23      0.34      1982
    ≥ 7 days       0.40      0.79      0.53      1299

    accuracy                           0.45      3281
   macro avg       0.51      0.51      0.43      3281
weighted avg       0.53      0.45      0.41      3281

Confusion Matrix:
 [[ 456 1526]
 [ 277 1022]]
ROC-AUC: 0.5413


In [31]:
# --- 0) make sure the dictionary table is loaded (if not already)
d_icd_diagnoses = pd.read_csv("d_icd_diagnoses.csv/d_icd_diagnoses.csv")

# --- 1) MI admissions lookup (admission-level)
mi_keys = set(zip(MI_finaldf["subject_id"], MI_finaldf["hadm_id"]))

# --- 2) quick dict: (icd_code, icd_version) -> long_title
d_icd_diagnoses["icd_code"] = d_icd_diagnoses["icd_code"].astype(str).str.replace(".", "", regex=False)
icd_map = dict(zip(
    zip(d_icd_diagnoses["icd_code"], d_icd_diagnoses["icd_version"]),
    d_icd_diagnoses["long_title"]
))

# --- 3) count top other diagnoses (non-MI) among MI admissions (chunked)
from collections import Counter
title_counts = Counter()

for chunk in pd.read_csv(
    "diagnoses_icd.csv.gz",
    compression="gzip",
    usecols=["subject_id", "hadm_id", "icd_code", "icd_version"],
    dtype={"icd_code": "string"},
    chunksize=500_000
):
    chunk["icd_code"] = chunk["icd_code"].str.replace(".", "", regex=False)

    # keep only MI admissions
    in_mi = list(zip(chunk["subject_id"], chunk["hadm_id"]))
    sub = chunk[[k in mi_keys for k in in_mi]].copy()
    if sub.empty:
        continue

    # remove MI codes
    is_mi = (
        ((sub["icd_version"] == 9) & sub["icd_code"].str.startswith("410")) |
        ((sub["icd_version"] == 10) & (sub["icd_code"].str.startswith("I21") | sub["icd_code"].str.startswith("I22")))
    )
    sub = sub[~is_mi]
    if sub.empty:
        continue

    # map to titles + count
    for code, ver in zip(sub["icd_code"].astype(str), sub["icd_version"].astype(int)):
        title_counts[icd_map.get((code, ver), f"UNKNOWN_{ver}_{code}")] += 1

# --- 4) print top 20
print("\n===== Top 20 Other Diagnoses (non-MI) in MI admissions =====")
for title, cnt in title_counts.most_common(20):
    print(f"{cnt:>6}  {title}")


===== Top 20 Other Diagnoses (non-MI) in MI admissions =====
  5822  Hyperlipidemia, unspecified
  5420  Atherosclerotic heart disease of native coronary artery without angina pectoris
  4883  Coronary atherosclerosis of native coronary artery
  4672  Acute kidney failure, unspecified
  3646  Personal history of nicotine dependence
  3431  Other and unspecified hyperlipidemia
  3318  Unspecified essential hypertension
  2884  Congestive heart failure, unspecified
  2780  Essential (primary) hypertension
  2700  Gastro-esophageal reflux disease without esophagitis
  2504  Acute posthemorrhagic anemia
  2289  Old myocardial infarction
  2185  Chronic kidney disease, unspecified
  2105  Acidosis
  2038  Anemia, unspecified
  2037  Long term (current) use of insulin
  1941  Type 2 diabetes mellitus with diabetic chronic kidney disease
  1854  Long term (current) use of antithrombotics/antiplatelets
  1833  Urinary tract infection, site not specified
  1828  Do not resuscitate
